# How-to
This notebook serves as a practical guide to common questions users might have.

**Table of content**

1. [Execute moldrug from the command line](#Execute-moldrug-from-the-command-line).
2. Create your own cost function.

In [21]:
# Import section
from moldrug.data import ligands, boxes, receptors
import tempfile, os, requests,inspect, gzip, shutil, yaml
from moldrug import utils

## Execute moldrug from the command line

MolDrug it is meant to be for using as a python module. However it has capabilities to work from the command line. Due to the complexity of the input for a normal simulation, [yaml](https://yaml.org/) structured file are used for the input specification. Let first see the help of moldrug.

In [2]:
! moldrug -h

usage: moldrug [-h] [-v] [-f FITNESS] [-o OUTDIR] yaml_file

For information of MolDrug:
    Docs: https://moldrug.readthedocs.io/en/latest/
    Source Code: https://github.com/ale94mleon/moldrug

positional arguments:
  yaml_file             The configuration yaml file

optional arguments:
  -h, --help            show this help message and exit
  -v, --version         show program's version number and exit
  -f FITNESS, --fitness FITNESS
                        The path to the user-custom fitness module; inside of which the given custom cost function must be implemented. See the docs for how to do it properly. E.g. my/awesome/fitness_module.py.By default will look in the moldrug.fitness module.
  -o OUTDIR, --outdir OUTDIR
                        The path to where all the files should be written. By default the current working directory will be used (where the command line was invoked).


As it shows only one positional arguments is needed `yaml_file`. Then you could give:

-   `fitness`: A customized fitness python module where your cost function must be implemented

-   `outdir`: The out directory.

Lets go steep by steep


### yaml_file anatomy

In [19]:
# Getting the data
tmp_path = tempfile.TemporaryDirectory()
lig = ligands.r_x0161
box = boxes.r_x0161
with open(os.path.join(tmp_path.name, 'x0161.pdbqt'), 'w') as f:
    f.write(receptors.r_x0161)


# Getting the CReM data base

url = "http://www.qsar4u.com/files/cremdb/replacements02_sc2.db.gz"
r = requests.get(url, allow_redirects=True)
crem_dbgz_path = os.path.join(tmp_path.name,'crem.db.gz')
crem_db_path = os.path.join(tmp_path.name,'crem.db')
open(crem_dbgz_path, 'wb').write(r.content)
with gzip.open(crem_dbgz_path, 'rb') as f_in:
    with open(crem_db_path, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

print(lig)
print(box)
print(tmp_path.name, os.listdir(tmp_path.name))

COC(=O)C=1C=CC(=CC1)S(=O)(=O)N
{'A': {'boxcenter': [12.11, 1.84, 23.56], 'boxsize': [22.5, 22.5, 22.5]}}
/tmp/tmpqxdyj58y ['crem.db.gz', 'x0161.pdbqt', 'crem.db']


Now we have all that we need. The SMILES, the definition of the box and the receptor. But first let see what are the arguments that hte function `utils.Local` needs:

In [16]:
inspect.getfullargspec(utils.Local.__init__).args

['self',
 'mol',
 'crem_db_path',
 'costfunc',
 'grow_crem_kwargs',
 'costfunc_kwargs']

We will pass all of this parameters using config.yaml file.

**Warning!**: Check your correct temporal file and change it accordantly. 


Here is an example yaml file to work with the class `utils.Local`.

```yaml
main:
  type: Local
  njobs: 1
  pick: 1
  mol: COC(=O)C=1C=CC(=CC1)S(=O)(=O)N
  costfunc: Cost
  costfunc_kwargs:
    vina_executable: vina
    receptor_path: /tmp/tmpqxdyj58y/x0161.pdbqt
    boxcenter:
      - 12.11
      - 1.84
      - 23.56
    boxsize:
      - 22.5
      - 22.5
      - 22.5
    exhaustiveness: 4
    ncores: 12
    num_modes: 1
  crem_db_path: /tmp/tmpqxdyj58y/crem.db
  grow_crem_kwargs:
    radius: 3
    min_atoms: 1
    max_atoms: 2
    ncores: 12
```
The structure of a yaml file is like a python dictionary but more human readable (see this [youtube video](https://www.youtube.com/watch?v=1uFVr15xDGg&list=PL6ebkIZFT4xXiVdpOeKR4o_sKLSY0aQf_&index=11) if you are not familiar).

First we have the directrix:

```yaml
main:
```

This is just the name of your main project (we will se that `moldrug.utils.GA` accept also follow projects) and could be any word. Then we have:

```yaml
  type: Local
  njobs: 1
  pick: 1
```

Looks how this section is inside of `main` (one indentation level). `type` is the name of the inside `moldrug.utils`; for now could be GA or Local (case insensitive). In this case we want to use the class `Local`. `njobs` and `pick` are the parameters when the class `Local` (or `GA`) is call. The rest of the parameters are just the parameters that needs `Local` for the initialization. All of them are at the same level of the previous ones. Because `costfunc_kwargs` is a dictionary; we add for its value a new indentation level:

```yaml
  costfunc_kwargs:
    vina_executable: vina
    receptor_path: /tmp/tmpqxdyj58y/x0161.pdbqt
    boxcenter:
      - 12.11
      - 1.84
      - 23.56
    boxsize:
      - 22.5
      - 22.5
      - 22.5
    exhaustiveness: 4
    ncores: 12
    num_modes: 1
```
As you could imagine tha keyword boxcenter is a list and this is one of the way to represent list in yaml. Cool right?

The next dictionary is the same that the yaml file and we will save it in the temporal file that we created.

In [22]:
config = {
    "main": {
        "type": "Local",
        "njobs": 1,
        "pick": 1,
        "mol": lig,
        "costfunc": "Cost",
        "costfunc_kwargs": {
            "vina_executable": "vina",
            "receptor_path": os.path.join(tmp_path.name, 'x0161.pdbqt'),
            "boxcenter": [
                12.11,
                1.84,
                23.56
            ],
            "boxsize": [
                22.5,
                22.5,
                22.5
            ],
            "exhaustiveness": 4,
            "ncores": 12,
            "num_modes": 1
        },
        "crem_db_path": os.path.join(tmp_path.name, 'crem.db'),
        "grow_crem_kwargs": {
            "radius": 3,
            "min_atoms": 1,
            "max_atoms": 2,
            "ncores": 12
        }
    }
}

# Save the config as a yaml file
with open(os.path.join(tmp_path.name, 'config.yml'), 'w') as f:
    yaml.dump(config, f)
os.listdir(tmp_path.name)

['crem.db.gz', 'x0161.pdbqt', 'config.yml', 'crem.db']

### Move to the directory and run moldrug

In [24]:
cwd = os.getcwd()
os.chdir(tmp_path.name)
! moldrug config.yml
os.chdir(cwd)
os.listdir(tmp_path.name)

The main job is being executed.
Calculating cost function...
100%|█████████████████████████████████████████████| 2/2 [00:06<00:00,  3.07s/it]
File local_pop.sdf was createad!
The main job finished!.


['local_result.pbz2',
 'crem.db.gz',
 'x0161.pdbqt',
 'config.yml',
 'local_pop.sdf',
 'crem.db']

As you see the simulation run successfully and the files:

1.  `local_result.pbz2`; the compresses version of the `Local` class with all the information of the simulation (binary file).

2.  `local_pop.sdf`; the mol structures. This file is useful to use inside Pymol.